# Note
## Jun. 6
>Status
1. By now, the best regressor is simple LassoCV using CV = 5
2. For categorical feature, use OneHot encoder.
3. The features having unique values, are dropped.
4. Try box-cox change of Y, but no imporvement.

>TODO
1. More work on feature engineering, try to reduce the dimension:
 + PCA
 + delete the dimensions with small variance

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
train_df = pd.read_csv('train.csv')
train_y = train_df.y
train_x = train_df.drop(['ID','y'],axis = 1)
test_df = pd.read_csv('test.csv')
test_x =  test_df.drop(['ID'],axis = 1)
cat_cols = train_x.select_dtypes(['object']).columns

# Compare training set and test set

# Feature engineering
## Issue: Categorical value of train and test are different
Sol:  try to change the catogorical data in test set to values existant in training set.

In [3]:
#for col in cat_cols:
#    print col, set(test_df[col].unique()).difference(set(train_x[col].unique()))

In [4]:
#x0_tran = {'ae':'ad', 'ag':'af','bb':'ba','an':'am','p':'r','av':'aw'}
#x2_tran = {'ab':'ac', 'ad':'ae', 'aj':'ai', 'u':'t', 'w':'x', 'ax':'ay'}
#x5_tran = {'a':'aa', 'b':'c', 't':'r', 'z':'y'}

#test_x['X0'] = map(lambda x: x0_tran[x] if x in x0_tran.keys() else x, test_x['X0'])
#test_x['X2'] = map(lambda x: x2_tran[x] if x in x2_tran.keys() else x, test_x['X2'])
#test_x['X5'] = map(lambda x: x5_tran[x] if x in x5_tran.keys() else x, test_x['X5'])

## Encode the categorical data

In [5]:
from sklearn.preprocessing import LabelBinarizer
# fit the Label binarizer
lbs = list()
for col in cat_cols:
    lb = LabelBinarizer().fit(train_x[col])
    lbs.append(lb)

# transform the trianing set
for col, lb in zip(cat_cols,lbs):
    bina_df = pd.DataFrame(lb.transform(train_x[col]),
                           columns= map(lambda x: col + '_' + str(x), range(len(lb.classes_))))
    train_x = pd.concat([train_x, bina_df], axis=1)

# transform the test set    
for col,lb in zip(cat_cols, lbs):
    bina_df = pd.DataFrame(lb.transform(test_x[col]),
                           columns= map(lambda x: col + '_' + str(x), range(len(lb.classes_))))
    bina_df.loc[bina_df.sum(axis = 1) == 0] =1./len(lb.classes_)
    test_x = pd.concat([test_x, bina_df], axis=1)
# drop the categorical data
train_x.drop(cat_cols, axis =1, inplace=True)
test_x.drop(cat_cols, axis =1, inplace=True)

In [6]:
print train_x.shape, test_x.shape

(4209, 563) (4209, 563)


# drop features which are collineare.

In [7]:
# if all the values are the same, then drop all
for col in [col  for col in train_x.columns if len(np.unique(train_x[col]))==1 ]:
    train_x.drop(col, axis = 1, inplace= True)
    test_x.drop(col, axis =1, inplace=True)

In [8]:
train_corr = train_x.corr()
colli = list()
for col in range(train_corr.shape[0]):
    for index in range(col):
        if train_corr.iloc[index, col] == 1:
            colli.append([index, col])
def get_group(collis):
    groups = list()
    for coll in colli:
        ctr = True
        for i in range(len(groups)):
            if coll[0] in groups[i] or coll[1] in groups[i]:
                groups[i] = groups[i]|{coll[0], coll[1]}
                ctr = False
        if ctr:
            groups.append({coll[0], coll[1]})
    return groups
groups = get_group(colli)
for group in groups:
    col = 'C_'+ '_'.join(map(lambda x: str(x), list(group)))
    train_x[col] = train_x.iloc[:,list(group)].mean(axis=1)
    test_x[col] = test_x.iloc[:,list(group)].mean(axis=1)
cols_drop = list()
for group in groups:
    cols_drop.append(train_x.columns[list(group)])
for col in cols_drop:
    train_x.drop(col, axis = 1, inplace=True)
    test_x.drop(col, axis = 1, inplace=True)
print train_x.shape, test_x.shape

(4209, 478) (4209, 478)


## Use PCA

In [9]:
from sklearn.decomposition import PCA
if False:
    pca = PCA(n_components=136)
    pca.fit(train_x)
    train_x = pca.transform(train_x)
    test_x = pca.transform(test_x)

pca_cumsum = np.cumsum(pca.explained_variance_ratio_)
for i in range(200):
    if pca_cumsum[i] > 0.95:
        print i
        break

In [12]:
train_x.to_pickle('train_x.pkl')
train_y.to_pickle('train_y.pkl')
test_x.to_pickle('test_x.pkl')